# Create Minds Matter Calendar events 

This notebook will read a computer-readable google sheet of calendar events and programmatically create each event in the Google calendar of the Minds Matter Admin account. It will also invite the relevant groups to each event. 

**Warning: You must complete the pre-work in the [How to run the calendar script](https://docs.google.com/document/d/10Kf_lHase22wk-F24BV5QlQ2uinig5b5E7TwfCtG1wI/edit?usp=sharing) google doc before you can run this notebook!!**

If you've completed the pre-work, proceed to filling in the noteobook inputs and running the cells:

### Step 1: Fill in manual inputs

In [1]:
## FILL IN MANUAL INPUTS ##

# copy & paste the Spreadsheet ID from the pre-work steps here
SPREADSHEET_ID = "1FlSVcsv0BxdOET-mlgMfgYckGWHMYVOaNBwFI-tfxJs"

In [29]:
from __future__ import print_function
import datetime
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
import numpy as np
import os.path
import pickle
import time

import pandas as pd

from helper_functions import * 

In [3]:
print(f"This notebook was run on {datetime.datetime.today()}")

This notebook was run on 2024-11-17 11:48:24.340834


### Step 2: Authenticate your account

Running the following cells will result in a pop-up window allowing you to authenticate and store the credentials in a `token.pickle` file. A "warning" may pop up telling you that Google hasn't verified this app. Click "Advanced" and then "Go to Untitled Project." Select all of the items that you need to access and click "Continue."

Once you have finished authenticating, a `token.pickle` file will get dropped into your working directory.

In [4]:
creds = read_credentials()

Reading token...
Successfully loaded credentials!


In [5]:
# build the calendar service
service = build("calendar", "v3", credentials=creds)

Test that the authentication has worked by call the Calendar API and printing the next 10 events in the user's calendar. Note: You may not have any upcoming events, that is fine. If this cell runs successfully, the Google API is working as intended. 

In [6]:
now = datetime.datetime.utcnow().isoformat() + "Z" # 'Z' indicates UTC time
print('Getting the upcoming 10 events')
events_result = service.events().list(calendarId=CALENDER_ID, timeMin=now,
                                    maxResults=10, singleEvents=True,
                                    orderBy='startTime').execute()
events = events_result.get('items', [])

if not events:
    print('No upcoming events found.')
for event in events:
    start = event['start'].get('dateTime', event['start'].get('date'))
    print(start, event['summary'])

/tmp/ipykernel_1488/3807546399.py:1: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  now = datetime.datetime.utcnow().isoformat() + "Z" # 'Z' indicates UTC time


Getting the upcoming 10 events
2024-11-23T10:00:00-08:00 Test Prep
2024-11-23T10:00:00-08:00 Writing and Critical Thinking
2024-11-23T10:00:00-08:00 Test Prep
2024-11-23T10:00:00-08:00 Writing and Critical Thinking
2024-11-23T10:00:00-08:00 Senior Enrichment
2024-11-23T10:00:00-08:00 Test Prep
2024-11-23T10:00:00-08:00 Senior Enrichment
2024-11-23T10:00:00-08:00 Senior Enrichment
2024-11-23T10:00:00-08:00 Writing and Critical Thinking
2024-11-23T11:00:00-08:00 Math Matters


### 3. Read data from the Academic calendar--Computer readable Google sheet

In [12]:
def pull_sheet_data(SPREADSHEET_ID, RANGE_NAME):
    
    service = build('sheets', 'v4', credentials=creds)
    sheet = service.spreadsheets()
    result = sheet.values().get(
        spreadsheetId=SPREADSHEET_ID,
        range=RANGE_NAME).execute()
    values = result.get('values', [])
    
    if not values:
        print('No data found.')
    else:
        rows = sheet.values().get(spreadsheetId=SPREADSHEET_ID,
                                  range=RANGE_NAME).execute()
        data = rows.get('values')
        print("COMPLETE: Data copied")
        return data
data = pull_sheet_data(SPREADSHEET_ID , 'Computer_Readable!A1:K')

COMPLETE: Data copied


In [13]:
# sanity check 
print(len(data))
data[1]

208


['08/17/2024',
 '11:00 AM - 01:00 PM',
 'new students',
 'students2027',
 'New Student Orientation2',
 '1',
 '11:00 AM',
 '1:00 PM',
 '3gkh5l9t8vaag0fonqg8st8vfo',
 '',
 '11:00 AM - 01:00 PM']

In [14]:
df = pd.DataFrame(data[1:], columns=data[0])

In [15]:
# sanity check
df.head()

,Date,Time,Calendar Assignment,Share with group,Event Title,Color,start,end,calendarEventID,Notes,Time Range Formula
0,08/17/2024,11:00 AM - 01:00 PM,new students,students2027,New Student Orientation2,1,11:00 AM,1:00 PM,3gkh5l9t8vaag0fonqg8st8vfo,,11:00 AM - 01:00 PM
1,08/18/2024,11:00 AM - 01:00 PM,ec,ec,EC Fall Retreat,1,11:00 AM,1:00 PM,l9dls4v2r91fd588gk67imm800,,11:00 AM - 01:00 PM
2,08/24/2024,01:00 PM - 03:30 PM,volunteers,volunteers,Volunteer Orientation and Training,1,1:00 PM,3:30 PM,62ik7i4i5ulkdlc7b708pf53vs,,01:00 PM - 03:30 PM
3,09/07/2024,10:00 AM - 01:00 PM,ec,ec,Kick-off,1,10:00 AM,1:00 PM,jaudf3kt586qa3hiflm75dpufg,,10:00 AM - 01:00 PM
4,09/07/2024,10:00 AM - 01:00 PM,WCT,wct-instructors,Kick-off,1,10:00 AM,1:00 PM,4au3mqsf5f6aotqc29nh0te0r4,,10:00 AM - 01:00 PM


### 4. Create events

Note this will take a few minutes!

In [ ]:
eventservice=service.events()

In [21]:
# create events
for k,row in df.iterrows():
    body={
        "description": row['Event Title'],
        "summary":  row['Event Title'],
        "start":{
            "dateTime": datetime.datetime.strptime(row.Date + " " + row.start, "%m/%d/%Y %I:%M %p").isoformat(),
            "timeZone": "America/Los_Angeles"
        },
        "end":{
            "dateTime": datetime.datetime.strptime(row.Date + " " + row.end, "%m/%d/%Y %I:%M %p").isoformat(),
            "timeZone": "America/Los_Angeles"
        },
        "colorId": f"{row.Color}",
        "guestsCanInviteOthers": False

    }
    
    if row.calendarEventID:
        pass
    else:
        r=eventservice.insert(calendarId=CALENDER_ID,
                             body=body)
        response=r.execute()
        df.loc[k, 'calendarEventID']=response["id"]
        time.sleep(.2)

In [28]:
for k,row in df.iterrows():
    r = eventservice.delete(calendarId=CALENDER_ID, eventId=row.calendarEventID)
    response=r.execute()
    print(response)

HttpError: <HttpError 403 when requesting https://www.googleapis.com/calendar/v3/calendars/c_822m07aktpjne3gfbr7cj8gjks%40group.calendar.google.com/events/3gkh5l9t8vaag0fonqg8st8vfo? returned "Forbidden". Details: "[{'domain': 'global', 'reason': 'forbidden', 'message': 'Forbidden'}]">

Spot check that events were created by going into the Admin Google Calendar and checking a few to see if they show up.

### 5. Invite user groups to events

Note this will take a few minutes!

In [17]:
for k,row in df.iterrows():
    emails=map(lambda x: x.replace(' ','') ,row['Share with group'].split(','))
    event = eventservice.get(calendarId=CALENDER_ID, eventId=row.calendarEventID).execute()
    attendees=event.get('attendees',[])
    for email in emails:
        try:
            invite=next(a for a in attendees if a['email']==email + "@mindsmatterseattle.org")
        except StopIteration:
            print(f'inviting {email} to {event["summary"]}')
            invite = {
                'email': f'{email}@mindsmatterseattle.org'
            }
            attendees.append(invite)

    event['attendees'] =  attendees 
    eventservice.update(calendarId=CALENDER_ID, eventId=row.calendarEventID, body=event).execute()

inviting students2027 to New Student Orientation
inviting ec to EC Fall Retreat
inviting volunteers to Volunteer Orientation and Training
inviting ec to Kick-off
inviting wct-instructors to Kick-off
inviting math-instructors to Kick-off
inviting testprep-instructors to Kick-off
inviting senior-enrichment-instructors to Kick-off
inviting 2027mentors to Kick-off
inviting students2027 to Kick-off
inviting 2026mentors to Kick-off
inviting students2026 to Kick-off
inviting 2025mentors to Kick-off
inviting students2025 to Kick-off
inviting college-counseling to Kick-off
inviting wct-instructors to Writing and Critical Thinking
inviting students2027 to Writing and Critical Thinking
inviting math-instructors to Math Matters
inviting students2027 to Math Matters
inviting testprep-instructors to Test Prep
inviting students2026 to Test Prep
inviting senior-enrichment-instructors to Senior Enrichment
inviting students2025 to Senior Enrichment
inviting collegecounseling to Senior Enrichment
invitin

### 6 (Optional): Save output for historical records

In [18]:
df.to_csv("Ren_second_run_2024.csv", index=False)

In [ ]:
# delete all events loaded in df
for k,row in df.iterrows():
    try {
      CalendarApp.getCalendarById(CALENDER_ID).getEventById(row.calendarEventID).deleteEvent();
    } catch(e) {
      console.error("Error deleting event " + delete_eventid + " from calendar " + calendarId + ". Error: " + e);
    }